In [34]:
using Distributions, Random, GLM, StatsBase, DataFrames, HypothesisTests;
Random.seed!(2343)

MersenneTwister(2343)

In [25]:
#treatments(1 = medication, 2 = chemotherapy, 3 = herbal remedies)
treatment = rand(1:3, 30)
size(treatment)

(30,)

In [26]:
#outcome after treatment 6 weeks trial phase (1 = worse, 2 = recovered, 3 = deceased)
outcome = rand(1:3, 30)
size(outcome)

(30,)

In [27]:
#cell size of the cancer 
cellsize = rand(Normal(), 30) * 100
size(cellsize)

(30,)

In [28]:
data = DataFrame(Treatment = treatment, Outcome = outcome, GrowthRate = cellsize)
describe(data)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Real,Float64,Real,Int64,DataType
1,Treatment,1.76667,1,2.0,3,0,Int64
2,Outcome,1.86667,1,2.0,3,0,Int64
3,GrowthRate,20.6999,-139.232,13.362,200.953,0,Float64


In [29]:
testing_set = data[16:end, :]

,Treatment,Outcome,GrowthRate
,Int64,Int64,Float64
1,2,1,-69.0988
2,1,2,-58.3271
3,1,3,119.378
4,2,2,134.45
5,1,1,-80.7284
6,1,2,24.0926
7,1,2,52.9137
8,1,2,81.2065
9,3,2,-95.7767


In [30]:
training_data = data[1:15, :]

,Treatment,Outcome,GrowthRate
,Int64,Int64,Float64
1,3,1,9.51982
2,1,2,-139.232
3,1,1,-32.8355
4,1,2,-48.8165
5,3,2,23.6783
6,3,2,46.2846
7,3,1,-2.44624
8,3,2,-91.3068
9,2,3,3.70816


In [31]:
data

,Treatment,Outcome,GrowthRate
,Int64,Int64,Float64
1,3,1,9.51982
2,1,2,-139.232
3,1,1,-32.8355
4,1,2,-48.8165
5,3,2,23.6783
6,3,2,46.2846
7,3,1,-2.44624
8,3,2,-91.3068
9,2,3,3.70816


In [36]:
#fit training data in linear model
tm = lm(@formula(Treatment ~ Outcome + GrowthRate), training_data)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

Treatment ~ 1 + Outcome + GrowthRate

Coefficients:
────────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error     t  Pr(>|t|)    Lower 95%   Upper 95%
────────────────────────────────────────────────────────────────────────────
(Intercept)  1.87234       0.63428   2.95    0.0121   0.490361    3.25432
Outcome      0.103013      0.327466  0.31    0.7585  -0.610474    0.8165
GrowthRate   0.00191226    0.003012  0.63    0.5374  -0.00465033  0.00847484
────────────────────────────────────────────────────────────────────────────

In [38]:
dof_residual(tm) #cal residual degrees of freedom

12.0

In [58]:
predict(tm)

15-element Vector{Float64}:
 1.9935565635895665
 1.812118133000041
 1.9125623631806152
 1.9850158013633072
 2.1236443577638893
 2.1668734370535656
 1.9706744066650055
 1.9037633789180926
 2.188469447270655
 1.930538740098602
 2.238402700084193
 2.307958258115575
 1.7693768804843735
 2.388663390522228
 2.308382141890285

In [40]:
coef(tm) #cal Coef.

3-element Vector{Float64}:
 1.872339112568858
 0.10301312640696968
 0.0019122557531876703

In [42]:
vcov(tm) #est variance co-variance matrix coef

3×3 Matrix{Float64}:
  0.402311     -0.192141      0.000298009
 -0.192141      0.107234     -0.000189028
  0.000298009  -0.000189028   9.07214e-6

In [44]:
deviance(tm) #deviance of the coef

10.412167335621064

In [52]:
mean(predict(tm))

2.0666666666666664

In [51]:
mean(testing_set[:, :Treatment])

1.4666666666666666

In [53]:
mean(training_data[:, :Treatment])

2.066666666666667

In [54]:
mean(data[:, :Treatment])

1.7666666666666666

In [57]:
mean(predict(tm)) - mean(testing_set[:, :Treatment]) * 100 / 2

-71.26666666666667

In [61]:
cancermodel = lm(@formula(Treatment ~ Outcome + GrowthRate), data)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

Treatment ~ 1 + Outcome + GrowthRate

Coefficients:
───────────────────────────────────────────────────────────────────────────────
                    Coef.  Std. Error      t  Pr(>|t|)    Lower 95%   Upper 95%
───────────────────────────────────────────────────────────────────────────────
(Intercept)   1.61038       0.475357    3.39    0.0022   0.635023    2.58573
Outcome       0.085106      0.248057    0.34    0.7342  -0.423865    0.594077
GrowthRate   -0.000124343   0.0019447  -0.06    0.9495  -0.00411453  0.00386585
───────────────────────────────────────────────────────────────────────────────

In [62]:
predict(cancermodel)

30-element Vector{Float64}:
 1.6942983202515345
 1.7979006227263996
 1.6995649283817564
 1.7866580366980083
 1.7776437828559575
 1.774832833191667
 1.6957862212751562
 1.7919414355894048
 1.8652329208009226
 1.6983960232953406
 1.7701816779691475
 1.8574632195199794
 1.708875494119676
 ⋮
 1.7638700798937
 1.7055200981085785
 1.777592268846721
 1.7740085503989689
 1.770490522419635
 1.7924972400984172
 1.6950899471259964
 1.8496703653312636
 1.7807594844664665
 1.7784487986517432
 1.7688691630943503
 1.7556008586813425

In [63]:
mean(predict(cancermodel))

1.7666666666666666